In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import train_test_split

C:\Users\iramaniparth\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\iramaniparth\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [17]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
from scipy.stats import skew, boxcox
from math import exp, log
import xgboost as xgb

In [2]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [18]:
def scale_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler

In [3]:
train = pd.read_csv('amex.csv')
train.drop(['Unnamed: 0','mvar16','mvar19','mvar17'],axis=1,inplace=True)
train.head()
test=pd.read_csv('amextest.csv')
test.drop(['Unnamed: 0','mvar16','mvar19','mvar17'],axis=1,inplace=True)
train['mvar47'].loc[train['mvar47'] == 'L'] = 0
train['mvar47'].loc[train['mvar47'] == 'C'] = 1
test['mvar47'].loc[test['mvar47'] == 'L'] = 0
test['mvar47'].loc[test['mvar47'] == 'C'] = 1
kall = train.iloc[:,1:45]
kalltr = kall.dropna()

values = kall.values

from sklearn.preprocessing import Imputer
imputer = Imputer()

transformed_values = imputer.fit_transform(values)

X = pd.DataFrame(transformed_values)
X.columns = kall.columns

for index in X.columns:
    train[index]=X[index]

C:\Users\iramaniparth\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
kall = test.iloc[:,1:45]
kalltr = kall.dropna()

values = kall.values

from sklearn.preprocessing import Imputer
imputer = Imputer()

transformed_values = imputer.fit_transform(values)

X = pd.DataFrame(transformed_values)
X.columns = kall.columns

for index in X.columns:
    test[index]=X[index]

In [5]:
train.drop(['application_key'], axis=1, inplace=True)
train_y = train['default_ind']
train.drop(['default_ind'], axis=1, inplace=True)
key = test['application_key']
test.drop(['application_key'], axis=1, inplace=True)

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train)

# Apply transform to both the training set and the test set.
train1 = scaler.transform(trains)
test1 = scaler.transform(tests)

In [44]:
from sklearn.decomposition import PCA

# Make an instance of the Model
pca = PCA(.95)

In [45]:
pca.fit(train1)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [46]:
train2 = pca.transform(train1)
test2 = pca.transform(test1)

trains = train2
tests = test2

In [10]:
train20=np.array(train2)
mn=np.mean(train20, axis=0)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train20, np.array(train_y), random_state=0, test_size = 0.25)
print(train20.shape)

(80000, 33)


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt1 = GradientBoostingClassifier(random_state=0,max_depth=4, learning_rate=0.1)
gbrt1.fit(X_train, y_train)
print("Accuracy on training set: {:.6f}".format(gbrt1.score(X_train, y_train)))
print("Accuracy on test set: {:.6f}".format(gbrt1.score(X_test, y_test)))

Accuracy on training set: 0.814467
Accuracy on test set: 0.807700


In [27]:
test20=np.array(test2)
ynew = gbrt1.predict_proba(test20)

d=pd.DataFrame()
d['application_key']=key
d['0']=ynew[:,0]
d['1']=ynew[:,1]

y=gbrt1.predict(test20)

d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_109.csv', header = False, index = False)

In [47]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

In [48]:
ITERATIONS = 5 # 1000
TRAINING_SIZE = 80000*0.75 # 20000000
TEST_SIZE = 80000*0.25

In [49]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx',
        early_stopping_rounds=10,
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

result_xgb = bayes_cv_tuner.fit(trains, np.array(train_y), callback=status_print)

Model #1
Best ROC-AUC: 0.7971
Best params: {'colsample_bylevel': 0.4160029192647807, 'colsample_bytree': 0.7304484857455519, 'gamma': 0.13031389926541354, 'learning_rate': 0.042815319280763466, 'max_delta_step': 13, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 87, 'reg_alpha': 5.497557739289786e-07, 'reg_lambda': 0.05936070635912049, 'scale_pos_weight': 0.060830282487222144, 'subsample': 0.13556548021189216}

Model #2
Best ROC-AUC: 0.8
Best params: {'colsample_bylevel': 0.8390144719977516, 'colsample_bytree': 0.8844821246070537, 'gamma': 4.358684608480795e-07, 'learning_rate': 0.7988179462781242, 'max_delta_step': 17, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 68, 'reg_alpha': 0.0005266983003701547, 'reg_lambda': 276.5424475574225, 'scale_pos_weight': 0.3016410771843142, 'subsample': 0.9923710598637134}

Model #3
Best ROC-AUC: 0.8
Best params: {'colsample_bylevel': 0.8390144719977516, 'colsample_bytree': 0.8844821246070537, 'gamma': 4.358684608480795e-07, 'learni

In [50]:
tests0=np.array(tests)
ynew = result_xgb.predict_proba(tests0)

d=pd.DataFrame()
d['application_key']=key
d['0']=ynew[:,0]
d['1']=ynew[:,1]

y=result_xgb.predict(tests0)

d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_113.csv', header = False, index = False)

C:\Users\iramaniparth\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [51]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMRegressor(
        objective='binary',
        metric='auc',
        n_jobs=1,
        verbose=0,
        use_missing=True
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (1, 100),      
        'max_depth': (0, 50),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'subsample_freq': (0, 10),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'subsample_for_bin': (100000, 500000),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 100),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result_lgb = bayes_cv_tuner.fit(trains, np.array(train_y), callback=status_print)

Model #1
Best ROC-AUC: 0.7925
Best params: {'colsample_bytree': 0.4160029192647807, 'learning_rate': 0.28539836866041823, 'max_bin': 940, 'max_depth': 16, 'min_child_samples': 34, 'min_child_weight': 4, 'n_estimators': 68, 'num_leaves': 74, 'reg_alpha': 5.497557739289786e-07, 'reg_lambda': 0.05936070635912049, 'scale_pos_weight': 0.060830282487222144, 'subsample': 0.13556548021189216, 'subsample_for_bin': 171234, 'subsample_freq': 6}

Model #2
Best ROC-AUC: 0.8045
Best params: {'colsample_bytree': 0.8390144719977516, 'learning_rate': 0.5842928269761146, 'max_bin': 373, 'max_depth': 48, 'min_child_samples': 43, 'min_child_weight': 1, 'n_estimators': 57, 'num_leaves': 36, 'reg_alpha': 0.0005266983003701547, 'reg_lambda': 276.5424475574225, 'scale_pos_weight': 0.3016410771843142, 'subsample': 0.9923710598637134, 'subsample_for_bin': 406716, 'subsample_freq': 4}

Model #3
Best ROC-AUC: 0.8045
Best params: {'colsample_bytree': 0.8390144719977516, 'learning_rate': 0.5842928269761146, 'max_bi

In [52]:
tests0=np.array(tests)
ynew = result_lgb.predict(tests0)

d=pd.DataFrame()
d['application_key']=key
d['1']=pd.DataFrame(ynew)
d['0']=1-d['1']

d.loc[d['1']>d['0'], 'pred'] = 1
d.loc[d['0']>d['1'], 'pred'] = 0

#y=result_lgb.predict(test0)

#d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_114.csv', header = False, index = False)

In [55]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = GradientBoostingClassifier(),
    search_spaces = {
        'learning_rate': (10**-5, 10, 'log-uniform'),     
        'max_depth': (0, 50),
        'subsample': (0.01, 1.0, 'uniform'),
        'n_estimators': (50, 100),
        'min_samples_leaf': (1, 100),
        'min_samples_split': (2, 100),
        'max_features': (1, 8),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result_gbm = bayes_cv_tuner.fit(trains, np.array(train_y), callback=status_print)

Model #1
Best ROC-AUC: 0.8042
Best params: {'learning_rate': 0.0028881766539144717, 'max_depth': 36, 'max_features': 8, 'min_samples_leaf': 32, 'min_samples_split': 68, 'n_estimators': 71, 'subsample': 0.35742202155015257}

Model #2
Best ROC-AUC: 0.8042
Best params: {'learning_rate': 0.0028881766539144717, 'max_depth': 36, 'max_features': 8, 'min_samples_leaf': 32, 'min_samples_split': 68, 'n_estimators': 71, 'subsample': 0.35742202155015257}

Model #3
Best ROC-AUC: 0.8042
Best params: {'learning_rate': 0.0028881766539144717, 'max_depth': 36, 'max_features': 8, 'min_samples_leaf': 32, 'min_samples_split': 68, 'n_estimators': 71, 'subsample': 0.35742202155015257}

Model #4
Best ROC-AUC: 0.8042
Best params: {'learning_rate': 0.0028881766539144717, 'max_depth': 36, 'max_features': 8, 'min_samples_leaf': 32, 'min_samples_split': 68, 'n_estimators': 71, 'subsample': 0.35742202155015257}

Model #5
Best ROC-AUC: 0.8042
Best params: {'learning_rate': 0.0028881766539144717, 'max_depth': 36, 'ma

In [56]:
tests0=np.array(tests)
ynew = result_gbm.predict_proba(tests0)

d=pd.DataFrame()
d['application_key']=key
d['0']=ynew[:,0]
d['1']=ynew[:,1]

y=result_gbm.predict(tests0)

d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_115.csv', header = False, index = False)

In [34]:
def load_data(train, test):
    ntrain = train.shape[0]
    ntest = test.shape[0]
    train_test = pd.concat((train, test)).reset_index(drop=True)
    numeric_feats = train_test.dtypes[train_test.dtypes != "object"].index

    # compute skew and do Box-Cox transformation
    skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
    print("\nSkew in numeric features:")
    print(skewed_feats)
    # transform features with skew > 0.25 (this can be varied to find optimal value)
    skewed_feats = skewed_feats[skewed_feats > 0.25]
    skewed_feats = skewed_feats.index
    for feats in skewed_feats:
        train_test[feats] = train_test[feats] + 1
        train_test[feats], lam = boxcox(train_test[feats])
    features = train.columns
    cats = [feat for feat in features if 'cat' in feat]
    # factorize categorical features
    for feat in cats:
        train_test[feat] = pd.factorize(train_test[feat], sort=True)[0]
    x_train = train_test.iloc[:ntrain, :]
    x_test = train_test.iloc[ntrain:, :]
    train_test_scaled, scaler = scale_data(train_test)
    train, _ = scale_data(x_train, scaler)
    test, _ = scale_data(x_test, scaler)


    return train, test

In [35]:
trains, tests = load_data(train, test)


Skew in numeric features:
mvar1       0.118939
mvar2       4.515416
mvar3       4.220594
mvar4       8.280500
mvar5      10.197799
mvar6       7.496327
mvar7      33.476509
mvar8       4.536171
mvar9       8.590508
mvar10     15.454673
mvar11     15.572524
mvar12      6.149522
mvar13     51.465922
mvar14    105.896380
mvar15      7.825176
mvar18      5.151784
mvar20      3.142167
mvar21      1.623566
mvar22      1.012030
mvar23      2.242447
mvar24     41.689452
mvar25      1.752506
mvar26      1.575906
mvar27      1.582929
mvar28      0.984432
mvar29      0.936213
mvar30      0.055905
mvar31      1.409096
mvar32      2.507065
mvar33      1.580553
mvar34      5.941473
mvar35      5.393156
mvar36      2.665494
mvar37      2.091497
mvar38      2.756444
mvar39     11.595155
mvar40      0.023986
mvar41     -1.718782
mvar42      1.193575
mvar43      1.652678
mvar44     -0.464856
mvar45      7.290179
mvar46      8.018833
mvar47     -0.619773
dtype: float64
